## 0) Environment and Path

In [1]:
# %pip install pandas numpy scikit-learn

import os
import re
import gc
import numpy as np
import pandas as pd

DATA_DIR = "./data"  # Change to the directory containing glathida_on_grid_RGIxx.csv
OUT_DIR  = "./output"  # Export results directory
os.makedirs(OUT_DIR, exist_ok=True)

import glob
paths = sorted(glob.glob(os.path.join(DATA_DIR, "glathida_on_grid_RGI*.csv")))
len(paths), paths[:3]

(16,
 ['./data/glathida_on_grid_RGI01.csv',
  './data/glathida_on_grid_RGI02.csv',
  './data/glathida_on_grid_RGI03.csv'])

## 1) Read and merge

In [2]:
def extract_rgi_from_name(path: str) -> str:
    """
    Extract RGI zone numbers from filenames.
    """
    m = re.search(r'RGI(\d+)', os.path.basename(path))
    if m:
        return f"RGI{m.group(1)}"
    return "RGI00"

dfs = []
for p in paths:
    rgi = extract_rgi_from_name(p)
    df = pd.read_csv(p)
    df["RGI"] = rgi
    dfs.append(df)
    del df
    gc.collect()

global_df = pd.concat(dfs, axis=0, ignore_index=True)
del dfs
gc.collect()

print(global_df.shape)
global_df.head()

(375027, 44)


,Unnamed: 0,ij_grid,latitude,longitude,thickness,thickness_uncertainty,x_proj,y_proj,i_grid,j_grid,...,glacier_max_elev,glacier_length,glacier_oggm_volume,glacier_reference_mb,glacier_aar,glacier_avg_temp,glacier_avg_prcpsol,glacier_avg_prcp,glacier_avg_meltingtemp,RGI
0,0,0032_0047,63.476617,-146.567303,176.888889,NaN,521558.978337,7.038767e+06,32,47,...,3020.2078,28400.0,48.29876,-621.6,0.49328,-10.929161,1870.663963,2685.774119,14.045242,RGI01
1,1,0033_0047,63.476140,-146.564018,267.307692,NaN,521723.006656,7.038715e+06,33,47,...,3020.2078,28400.0,48.29876,-621.6,0.49328,-10.929161,1870.663963,2685.774119,14.045242,RGI01
2,2,0034_0048,63.475576,-146.560138,368.538462,NaN,521916.759245,7.038654e+06,34,48,...,3020.2078,28400.0,48.29876,-621.6,0.49328,-10.929161,1870.663963,2685.774119,14.045242,RGI01
3,3,0035_0048,63.474990,-146.556108,407.714286,NaN,522118.040436,7.038590e+06,35,48,...,3020.2078,28400.0,48.29876,-621.6,0.49328,-10.929161,1870.663963,2685.774119,14.045242,RGI01
4,4,0036_0048,63.474404,-146.552077,424.230769,NaN,522319.346036,7.038526e+06,36,48,...,3020.2078,28400.0,48.29876,-621.6,0.49328,-10.929161,1870.663963,2685.774119,14.045242,RGI01


## 2) Basic Cleaning

In [3]:
THICK_COL = "thickness"
LAT_COL, LON_COL = "lat", "lon"
ID_COL = "glacier_id" if "glacier_id" in global_df.columns else None

df = global_df.copy()

# Retain only entries where thickness > 0
if THICK_COL in df.columns:
    df = df[df[THICK_COL].notna()]
    df = df[df[THICK_COL] > 0]
# Reasonable latitude and longitude ranges
if LAT_COL in df.columns:
    df = df[(df[LAT_COL].between(-90, 90))]
if LON_COL in df.columns:
    df = df[(df[LON_COL].between(-180, 180))]

# If year or date is present, default to retaining only entries ≥2005 (adjustable/removable as required)
MIN_YEAR = 2005
if "year" in df.columns:
    df["year"] = pd.to_numeric(df["year"], errors="coerce")
    df = df[df["year"].isna() | (df["year"] >= MIN_YEAR)]
elif "date" in df.columns:
    years = pd.to_datetime(df["date"], errors="coerce").dt.year
    df = df[years.isna() | (years >= MIN_YEAR)]

df = df.reset_index(drop=True)
print(df.shape)
df.head()

(287916, 44)


,Unnamed: 0,ij_grid,latitude,longitude,thickness,thickness_uncertainty,x_proj,y_proj,i_grid,j_grid,...,glacier_max_elev,glacier_length,glacier_oggm_volume,glacier_reference_mb,glacier_aar,glacier_avg_temp,glacier_avg_prcpsol,glacier_avg_prcp,glacier_avg_meltingtemp,RGI
0,269,0023_0097,61.196387,-149.046639,160.50,35.0,390002.432266,6.786384e+06,23,97,...,1930.6503,9804.0,4.069497,-948.5,0.252815,-6.668839,2205.643441,2744.982065,17.967193,RGI01
1,270,0024_0097,61.196783,-149.045513,160.00,35.0,390064.278165,6.786426e+06,24,97,...,1930.6503,9804.0,4.069497,-948.5,0.252815,-6.668839,2205.643441,2744.982065,17.967193,RGI01
2,271,0025_0096,61.197314,-149.044018,161.25,35.0,390146.484415,6.786483e+06,25,96,...,1930.6503,9804.0,4.069497,-948.5,0.252815,-6.668839,2205.643441,2744.982065,17.967193,RGI01
3,272,0026_0095,61.198003,-149.042191,156.00,35.0,390247.063895,6.786556e+06,26,95,...,1930.6503,9804.0,4.069497,-948.5,0.252815,-6.668839,2205.643441,2744.982065,17.967193,RGI01
4,273,0026_0096,61.197724,-149.042923,155.50,35.0,390206.731706,6.786527e+06,26,96,...,1930.6503,9804.0,4.069497,-948.5,0.252815,-6.668839,2205.643441,2744.982065,17.967193,RGI01


## 3) Missing value handling

In [4]:
# Selecting Numerical Features
drop_cols = {"RGI", THICK_COL, LAT_COL, LON_COL, "year", "date"}
if ID_COL: drop_cols.add(ID_COL)

num_df = df.select_dtypes(include=[np.number])
feat_cols = [c for c in num_df.columns if c not in drop_cols]

print("Numeric feature count:", len(feat_cols))
feat_cols[:10]

# First perform median within region filling
df_impute = df.copy()
if "RGI" not in df_impute.columns:
    df_impute["RGI"] = "RGI00"

# Calculate the median using the RGI and fill in the values
for col in feat_cols:
    med_by_rgi = df_impute.groupby("RGI")[col].transform("median")
    df_impute[col] = df_impute[col].fillna(med_by_rgi)

# For remaining missing values, use the global median as a fallback.
global_median = df_impute[feat_cols].median()
df_impute[feat_cols] = df_impute[feat_cols].fillna(global_median)

df_impute[feat_cols].isna().mean().sort_values(ascending=False).head(10)

Numeric feature count: 38


Unnamed: 0               0.0
latitude                 0.0
longitude                0.0
thickness_uncertainty    0.0
x_proj                   0.0
y_proj                   0.0
i_grid                   0.0
j_grid                   0.0
survey_id                0.0
topo                     0.0
dtype: float64

## 4) Export the base merged global dataset

In [5]:
global_all_path = os.path.join(OUT_DIR, "global_all.csv")
df_impute.to_csv(global_all_path, index=False)
global_all_path


'./output/global_all.csv'

## 5) Anomaly Detection Feature Matrix Preparation

In [6]:
from sklearn.preprocessing import StandardScaler

X = df_impute[feat_cols].copy()

scaler = StandardScaler()
X_std = scaler.fit_transform(X.values)

X.shape

(287916, 38)

## 6) Three-Model Anomaly Detection + Voting Method (IsolationForest + LOF + DBSCAN)

By default, anomalies are excluded only if at least two-thirds of models flag them as such. This threshold can be adjusted via VOTE_K (values 1–3).

Empirical defaults: IF contamination=0.03, LOF n_neighbors=35 (automatically contracts for small samples), DBSCAN eps=1.5, min_samples=20 (in the standardised feature space).

In [7]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN

VOTE_K = 2                 
IF_CONT = 0.03
LOF_K = 35
DBSCAN_EPS = 1.5
DBSCAN_MIN_SAMPLES = 20
RANDOM_STATE = 42

n = X_std.shape[0]
votes = np.zeros(n, dtype=int)

# Isolation Forest
ifor = IsolationForest(
    n_estimators=300,
    max_samples="auto",
    contamination=IF_CONT,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)
pred_if = ifor.fit_predict(X_std)   # -1: outlier, 1: inlier
votes += (pred_if == -1).astype(int)

# LOF
k = min(LOF_K, max(5, n // 3))
lof = LocalOutlierFactor(
    n_neighbors=k,
    contamination=IF_CONT,
    novelty=False,
    n_jobs=-1
)
pred_lof = lof.fit_predict(X_std)   # -1: outlier, 1: inlier
votes += (pred_lof == -1).astype(int)

# DBSCAN
db = DBSCAN(eps=DBSCAN_EPS, min_samples=DBSCAN_MIN_SAMPLES, n_jobs=-1)
labels = db.fit_predict(X_std)      # -1: noise
votes += (labels == -1).astype(int)

is_outlier = votes >= VOTE_K
keep_mask = ~is_outlier

print("Total:", n,
      "| outliers:", int(is_outlier.sum()),
      f"({is_outlier.mean()*100:.2f}%)")

Total: 287916 | outliers: 3358 (1.17%)


In [8]:
df_clean = df_impute.loc[keep_mask].reset_index(drop=True)

global_clean_path = os.path.join(OUT_DIR, "global_clean.csv")
df_clean.to_csv(global_clean_path, index=False)

global_all_path, global_clean_path

('./output/global_all.csv', './output/global_clean.csv')

In [9]:
df_outliers = df_impute.loc[is_outlier].reset_index(drop=True)

outlier_list_path = os.path.join(OUT_DIR, "global_outliers.csv")
df_outliers.to_csv(outlier_list_path, index=False)

print(f"Number of outlier samples: {len(df_outliers)} / {len(df_impute)} "
      f"({len(df_outliers)/len(df_impute)*100:.2f}%)")
print("Save path:", outlier_list_path)

df_outliers.head()

Number of outlier samples: 3358 / 287916 (1.17%)
Save path: ./output/global_outliers.csv


,Unnamed: 0,ij_grid,latitude,longitude,thickness,thickness_uncertainty,x_proj,y_proj,i_grid,j_grid,...,glacier_max_elev,glacier_length,glacier_oggm_volume,glacier_reference_mb,glacier_aar,glacier_avg_temp,glacier_avg_prcpsol,glacier_avg_prcp,glacier_avg_meltingtemp,RGI
0,642,0126_0053,61.348084,-148.147205,262.500000,35.0,438634.810140,6.802099e+06,126,53,...,3521.4646,48000.0,127.297364,-597.4,0.631109,-7.016374,3174.713028,4030.75856,16.643426,RGI01
1,643,0126_0054,61.346655,-148.145352,258.200000,35.0,438731.113964,6.801938e+06,126,54,...,3521.4646,48000.0,127.297364,-597.4,0.631109,-7.016374,3174.713028,4030.75856,16.643426,RGI01
2,644,0127_0054,61.345788,-148.144279,262.500000,35.0,438786.788191,6.801840e+06,127,54,...,3521.4646,48000.0,127.297364,-597.4,0.631109,-7.016374,3174.713028,4030.75856,16.643426,RGI01
3,645,0127_0055,61.344534,-148.142748,273.333333,35.0,438866.282916,6.801699e+06,127,55,...,3521.4646,48000.0,127.297364,-597.4,0.631109,-7.016374,3174.713028,4030.75856,16.643426,RGI01
4,646,0127_0056,61.343369,-148.141342,305.666667,35.0,438939.207527,6.801568e+06,127,56,...,3521.4646,48000.0,127.297364,-597.4,0.631109,-7.016374,3174.713028,4030.75856,16.643426,RGI01


In [10]:
def quick_summary(name, d):
    print(f"--- {name} ---")
    print("shape:", d.shape)
    print("thickness describe:")
    print(d[THICK_COL].describe())

quick_summary("Before (global_all)", df_impute)
quick_summary("After  (global_clean)", df_clean)

--- Before (global_all) ---
shape: (287916, 44)
thickness describe:
count    287916.000000
mean        265.188372
std         223.982927
min           0.066667
25%          73.500000
50%         211.250000
75%         408.000000
max        3151.000000
Name: thickness, dtype: float64
--- After  (global_clean) ---
shape: (284558, 44)
thickness describe:
count    284558.000000
mean        265.104708
std         223.911987
min           0.066667
25%          73.500000
50%         211.000000
75%         407.857143
max        3151.000000
Name: thickness, dtype: float64


In [11]:
# ==== Cell A1: write base config snapshot (YAML) ====
import os, yaml
from datetime import datetime

# Make sure a configs directory exists
os.makedirs("./output/configs", exist_ok=True)

# Base config just for snapshot/logging (does not override your pipeline)
BASE_CFG = {
    "seed": 42,
    "data": {
        "dir": "./output/",
        "input_global_all": "./output/global_all.csv",
        "input_global_clean": "./output/global_clean.csv",
        "id_col": "glacier_id",   # set to None if this column is not available
        "rgi_col": "RGI",
        "target_col": "thickness",
    },
    "processing": {
        "scaler": "standard",       # standard / none
        "imputer": "median_by_rgi",
        "outlier_vote_k": 2,
    },
    "logging": {
        "save_dir": "./output/runs/",
        "experiment_name": "exp_glathida_v1"
    },
    "cv": {
        "n_repeats": 5,
        "test_size": 0.2,
        "with_supervision": True,
        "group_by_glacier": False
    },
    "optuna": {
        "preset": "fast",
        "n_trials": {"xgb": 40, "lgb": 40, "cat": 40, "mlp": 40}
    },
    "models": {"use": ["xgb", "lgb", "cat", "mlp"]},
    "artifacts": {
        "save_scaler": True,
        "save_feature_list": True,
        "save_config_snapshot": True
    },
    "_snapshot_ts": datetime.now().isoformat(timespec="seconds")
}

with open("./output/configs/base.yaml", "w", encoding="utf-8") as f:
    yaml.safe_dump(BASE_CFG, f, allow_unicode=True)

print("Saved ./output/configs/base.yaml")


Saved ./output/configs/base.yaml


In [12]:
# ==== Cell A2: feature + RGI stats report ====
import os, json, hashlib
import pandas as pd
import numpy as np
import yaml
from datetime import datetime

# Load the base config
cfg = yaml.safe_load(open("./output/configs/base.yaml", "r", encoding="utf-8"))

# Prepare save directory for reports
save_root = os.path.join(cfg["logging"]["save_dir"], cfg["logging"]["experiment_name"])
os.makedirs(save_root, exist_ok=True)

all_path  = cfg["data"]["input_global_all"]
clean_path= cfg["data"]["input_global_clean"]
rgi_col   = cfg["data"]["rgi_col"]
y_col     = cfg["data"]["target_col"]

df_all = pd.read_csv(all_path)

# Column types and missing ratio
desc = pd.DataFrame({
    "dtype": df_all.dtypes.astype(str),
    "n_missing": df_all.isna().sum(),
    "missing_rate": df_all.isna().mean()
}).sort_values("missing_rate", ascending=False)

# Sample counts by RGI
by_rgi = df_all[rgi_col].value_counts().rename_axis(rgi_col).reset_index(name="n_rows")

# Compare clean vs all (removed rows per region)
delta = None
if os.path.exists(clean_path):
    df_clean = pd.read_csv(clean_path)
    s_before = df_all.groupby(rgi_col).size().rename("n_before")
    s_after  = df_clean.groupby(rgi_col).size().rename("n_after")
    delta = pd.concat([s_before, s_after], axis=1).fillna(0).astype(int)
    delta["removed"] = delta["n_before"] - delta["n_after"]
    delta["removed_rate"] = delta["removed"] / delta["n_before"].replace(0, np.nan)

# Create a simple processing log
fingerprint = hashlib.md5(pd.util.hash_pandas_object(df_all, index=False).values).hexdigest()
proc_log = {
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "data_fingerprint_md5": fingerprint,
    "rows": int(len(df_all)),
    "cols": int(df_all.shape[1]),
    "rgi_unique": int(df_all[rgi_col].nunique()),
    "target": y_col,
    "notes": "Preprocessing report snapshot"
}

# Save reports
desc.to_csv(os.path.join(save_root, "feature_report.csv"))
by_rgi.to_csv(os.path.join(save_root, "rgi_counts.csv"), index=False)
if delta is not None:
    delta.to_csv(os.path.join(save_root, "rgi_removed_summary.csv"))

with open(os.path.join(save_root, "processing_log.json"), "w", encoding="utf-8") as f:
    json.dump(proc_log, f, ensure_ascii=False, indent=2)

print("Saved reports to:", save_root)


Saved reports to: ./output/runs/exp_glathida_v1


In [13]:
# ==== Cell A3: artifacts snapshot (feature_list, scaler, config snapshot) ====
import os, json, yaml, joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

cfg = yaml.safe_load(open("./output/configs/base.yaml", "r", encoding="utf-8"))
save_root = os.path.join(cfg["logging"]["save_dir"], cfg["logging"]["experiment_name"])
os.makedirs(save_root, exist_ok=True)

df_all = pd.read_csv(cfg["data"]["input_global_all"])
rgi_col = cfg["data"]["rgi_col"]
y_col   = cfg["data"]["target_col"]
id_col  = cfg["data"].get("id_col") or None

# Exclude target, RGI, and glacier_id (if present)
exclude_cols = {c for c in [y_col, rgi_col, id_col] if c}
features = [c for c in df_all.columns if c not in exclude_cols]

# Save feature list
if cfg["artifacts"]["save_feature_list"]:
    with open(os.path.join(save_root, "feature_list.json"), "w") as f:
        json.dump(features, f)

# Save a scaler trained on numeric columns
if cfg["processing"]["scaler"] == "standard" and cfg["artifacts"]["save_scaler"]:
    num_cols = df_all[features].select_dtypes(include=[np.number]).columns.tolist()
    scaler = StandardScaler().fit(df_all[num_cols].fillna(0))
    joblib.dump(scaler, os.path.join(save_root, "scaler.pkl"))

# Save a config snapshot for reproducibility
if cfg["artifacts"]["save_config_snapshot"]:
    with open(os.path.join(save_root, "config_snapshot.yaml"), "w", encoding="utf-8") as f:
        yaml.safe_dump(cfg, f, allow_unicode=True)

print("Artifacts saved in:", save_root)


Artifacts saved in: ./output/runs/exp_glathida_v1
